# PyTorch Object Detection and Tracking

## Step 1: Download the object detection system

Download the code to run the object detection

In [0]:
%%shell

# Install pycocotools
git clone https://github.com/cfotache/pytorch_objectdetecttrack

Cloning into 'pytorch_objectdetecttrack'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 36 (delta 0), reused 0 (delta 0), pack-reused 33
Unpacking objects: 100% (36/36), done.


In [0]:
%ls -l
%cd pytorch_objectdetecttrack/
%ls -l

total 532
drwxr-xr-x 2 root root   4096 Oct 17 18:23 config/
-rw-r--r-- 1 root root  22960 Oct 29  2014 Harry_Potter_and_the_Order_of_the_Phoenix_poster.jpg
drwxr-xr-x 2 root root   4096 Oct 17 18:38 images/
-rw-r--r-- 1 root root  14374 Oct 17 18:22 models.py
-rw-r--r-- 1 root root   3735 Oct 17 18:22 object_tracker.py
drwxr-xr-x 2 root root   4096 Oct 17 18:26 __pycache__/
-rw-r--r-- 1 root root 451724 Oct 17 18:22 PyTorch_Object_Detection.ipynb
drwxr-xr-x 6 root root   4096 Oct 17 19:59 pytorch_objectdetecttrack/
-rw-r--r-- 1 root root   5192 Oct 17 18:22 PyTorch_Object_Tracking.ipynb
-rw-r--r-- 1 root root    701 Oct 17 18:22 README.md
-rw-r--r-- 1 root root  10271 Oct 17 18:22 sort.py
drwxr-xr-x 3 root root   4096 Oct 17 18:22 utils/
/content/pytorch_objectdetecttrack/pytorch_objectdetecttrack
total 500
drwxr-xr-x 2 root root   4096 Oct 17 19:59 config/
drwxr-xr-x 2 root root   4096 Oct 17 19:59 images/
-rw-r--r-- 1 root root  14374 Oct 17 19:59 models.py
-rw-r--r-- 1 root root   

Download the YoloV3 pretrained weights

In [0]:
%%shell 

ls -l ./config
cd ./config
bash ./download_weights.sh

## Step 2: Apply the object recognition on an image

Necessary imports.

In [0]:
from models import *
from utils import *

import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

Define the configuration path and initialize Darknet

### Question: Where is defined Darknet? What is it?

### Question: What is the COCO dataset? What are the labels?

In [0]:
config_path='./config/yolov3.cfg'
weights_path='./config/yolov3.weights'
class_path='./config/coco.names'
img_size=416
conf_thres=0.8
nms_thres=0.4

# Load model and weights
model = Darknet(config_path, img_size=img_size)
model.load_weights(weights_path)
model.cuda()
model.eval()
classes = utils.load_classes(class_path)
Tensor = torch.cuda.FloatTensor

Most of the following code deals with resizing the image to a square while maintaining its aspect ratio and padding the overflow

### Question: What is the goal of the resizing step?

In [0]:
def detect_image(img):
    # scale and pad image
    ratio = min(img_size/img.size[0], img_size/img.size[1])
    imw = round(img.size[0] * ratio)
    imh = round(img.size[1] * ratio)
    img_transforms = transforms.Compose([ transforms.Resize((imh, imw)),
         transforms.Pad((max(int((imh-imw)/2),0), max(int((imw-imh)/2),0), max(int((imh-imw)/2),0), max(int((imw-imh)/2),0)),
                        (128,128,128)),
         transforms.ToTensor(),
         ])
    
    # convert image to Tensor
    image_tensor = img_transforms(img).float()
    print(image_tensor.shape)
    image_tensor = image_tensor.unsqueeze_(0)
    input_img = image_tensor.type(Tensor)
    
    # run inference on the model and get detections
    with torch.no_grad():
        detections = model(input_img)
        detections = utils.non_max_suppression(detections, 80, conf_thres, nms_thres)
    return detections[0]

Print the directory with the image that can be tested.

In [0]:
%ls -l ./images


Load an arbitrary image

In [0]:
%%shell
wget https://upload.wikimedia.org/wikipedia/en/e/e7/Harry_Potter_and_the_Order_of_the_Phoenix_poster.jpg

The following cells load an image, get the detections, and then display it with the bounding boxes around detected objects. 

Most of the code deals with scaling and padding the image, as well as getting different colors for each detected class.

In [0]:
# load image and get detections
#img_path = "images/blueangels.jpg"
img_path = "images/Intersection-Counts.jpg"
img_path = "images/olympic-trials.jpg"
img_path = "Harry_Potter_and_the_Order_of_the_Phoenix_poster.jpg"
prev_time = time.time()
img = Image.open(img_path)
detections = detect_image(img)
inference_time = datetime.timedelta(seconds=time.time() - prev_time)
print ('Inference Time: %s' % (inference_time))



### Question: describe the content of "detections"

In [0]:
print(detections)

In [0]:
%ls -l

In [0]:
# Get bounding-box colors
cmap = plt.get_cmap('tab20b') # get a colormap
colors = [cmap(i) for i in np.linspace(0, 1, 20)]

img = np.array(img)
plt.figure()
fig, ax = plt.subplots(1, figsize=(12,9))
ax.imshow(img)

pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
unpad_h = img_size - pad_y
unpad_w = img_size - pad_x

if detections is not None:
    unique_labels = detections[:, -1].cpu().unique()
    n_cls_preds = len(unique_labels)
    bbox_colors = random.sample(colors, n_cls_preds)
    # browse detections and draw bounding boxes
    for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:
        box_h = ((y2 - y1) / unpad_h) * img.shape[0]
        box_w = ((x2 - x1) / unpad_w) * img.shape[1]
        y1 = ((y1 - pad_y // 2) / unpad_h) * img.shape[0]
        x1 = ((x1 - pad_x // 2) / unpad_w) * img.shape[1]
        color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]
        bbox = patches.Rectangle((x1, y1), box_w, box_h, linewidth=2, edgecolor=color, facecolor='none')
        ax.add_patch(bbox)
        plt.text(x1, y1, s=classes[int(cls_pred)], color='white', verticalalignment='top',
                bbox={'color': color, 'pad': 0})
plt.axis('off')
plt.show();

### Question: Test an image you will choose yourself